In [2]:
# !pip install pandas polars duckdb pyarrow

#Part 1: Data Ingestion

**1.** Programmatic Download

In [3]:
import requests
import os
from pathlib import Path

down_dir = Path("data/raw")
down_dir.mkdir(parents=True, exist_ok=True)

download = [
    {
        'url': 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet',
        'filename': down_dir / 'yellow_tripdata_2024-01.parquet'
    },
    {
        'url': 'https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv',
        'filename': down_dir / 'taxi_zone_lookup.csv'
    }
]

for file in download:
  print(f'Downloading {file['url']}...')

  response = requests.get(file['url'], stream=True)

  response.raise_for_status()

  with open(file['filename'], 'wb') as f:
    for chunk in response.iter_content(chunk_size=8192):
      f.write(chunk)

  print(f'Downloaded to {file['filename']}')

print('All downloads completed.')

Downloaded to data/raw/yellow_tripdata_2024-01.parquet
Downloaded to data/raw/taxi_zone_lookup.csv
All downloads completed.


The above uses a "requests" implementation to download and store necessary files.

**2.** Data validation

In [4]:
import polars as pl

#a) verifing expected columns exist
ex_cols = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance', 'fare_amount', 'tip_amount', 'total_amount', 'payment_type']

taxi_df = pl.read_parquet(down_dir / 'yellow_tripdata_2024-01.parquet')
zones_df = pl.read_csv(down_dir / 'taxi_zone_lookup.csv')
taxi_schema = taxi_df.schema

actual_cols = list(taxi_schema.keys())

missing_cols = [col for col in ex_cols if col not in actual_cols]

if not missing_cols:
  print(f'\nAll expected columns exist: {ex_cols}')
else:
  # d.a)
  print(f'\nDataset is missing the following expected coloumns: {missing_cols}')
  raise Exception('Missing expected columns.')

#b) checking for valid datatime type in datetime columns
date_cols = ['tpep_pickup_datetime', 'tpep_dropoff_datetime']

for col in date_cols:
  if col in taxi_schema:
    datet = taxi_schema[col]
    if datet == pl.Datetime:
      print(f'\ncolumn {col} is of {datet} type')
    else:
      print(f'\ncolumn {col} is not of datetime type')
      raise Exception(f'column {col} is not of datetime type')

taxi_df = taxi_df.select(ex_cols)
taxi_schema = taxi_df.schema

#c) Report total row count and print a summary to the console
print(f'\nNumber of rows in the dataset: {len(taxi_df):,}')
print(f'Number of columns in the dataset: {len(taxi_df.columns):,}')
print('\nColumn names and types:')
print(taxi_schema)


All expected columns exist: ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance', 'fare_amount', 'tip_amount', 'total_amount', 'payment_type']

column tpep_pickup_datetime is of Datetime(time_unit='ns', time_zone=None) type

column tpep_dropoff_datetime is of Datetime(time_unit='ns', time_zone=None) type

Number of rows in the dataset: 2,964,624
Number of columns in the dataset: 10

Column names and types:
Schema({'tpep_pickup_datetime': Datetime(time_unit='ns', time_zone=None), 'tpep_dropoff_datetime': Datetime(time_unit='ns', time_zone=None), 'PULocationID': Int32, 'DOLocationID': Int32, 'passenger_count': Int64, 'trip_distance': Float64, 'fare_amount': Float64, 'tip_amount': Float64, 'total_amount': Float64, 'payment_type': Int64})


**3.** File organisation (.gitignore)

In [5]:
# # Create .gitignore file in the root directory
# gitignore_content = """
# # Data directories
# data/
# data/
# **/*.parquet
# **/*.csv
# **/*.db
# sample_data/
# sample_data/

# # Python
# __pycache__/
# *.pyc
# .ipynb_checkpoints/

# # Colab specific
# .session/
# *.ipynb
# """

# # Write to .gitignore
# with open('.gitignore', 'w') as f:
#     f.write(gitignore_content)

# print(".gitignore file created")
# print(f"Location: {os.path.abspath('.gitignore')}")

# # Verify file exists
# if Path('.gitignore').exists():
#     print(".gitignore file exists")
#     print("\nContents:")
#     with open('.gitignore', 'r') as f:
#         print(f.read())

#Part 2: Data Transformation & Analysis

**4.** Data Cleaning

In [6]:
# e) removing null from critical columns
clean_df = taxi_df.filter(pl.col('tpep_pickup_datetime').is_not_null()& pl.col('tpep_dropoff_datetime').is_not_null()&
                          pl.col('PULocationID').is_not_null()& pl.col('DOLocationID').is_not_null()& pl.col('fare_amount').is_not_null())

# h.e) Rows removed
length = len(taxi_df)
print(f'Number of rows removed due to null values: {len(taxi_df) - len(clean_df):,}')

# f) removing invalid rows
clean_df = (clean_df
            .filter(pl.col('trip_distance') > 0)
            .filter(pl.col('trip_distance') <= 50)
            .filter(pl.col('fare_amount') > 0)
            .filter(pl.col('fare_amount') <= 500)
            )

# h.f) Rows removed
print(f'Number of rows removed due to invalid or outlier values: {length - len(clean_df):,}')
length = len(clean_df)

# g) removing invalid times
clean_df = clean_df.filter(pl.col('tpep_pickup_datetime') < pl.col('tpep_dropoff_datetime'))

# h.g) Rows removed
print(f'Number of rows removed due to invalid times: {length - len(clean_df):,}')

print(f'\nNumber of rows in cleaned dataset: {len(clean_df):,}')

Number of rows removed due to null values: 0
Number of rows removed due to invalid or outlier values: 95,307
Number of rows removed due to invalid times: 112

Number of rows in cleaned dataset: 2,869,205


**5.** Feature Engineering

In [7]:
# Adding new columns
enriched = clean_df.with_columns([
    # i) trip duration in minutes
    ((pl.col('tpep_dropoff_datetime')-pl.col('tpep_pickup_datetime')).dt.total_seconds() / 60).alias('trip_duration_minutes'),

])

enriched = enriched.with_columns([
    # j) trip speed
    pl.when(pl.col('trip_duration_minutes') > 0)
    .then((pl.col('trip_distance') / pl.col('trip_duration_minutes')))
    .otherwise(0)
    .alias('trip_speed_mph'),

    # k) pickup hour
    pl.col('tpep_pickup_datetime').dt.hour().alias('pickup_hour'),

    # l) pickup day
    pl.col('tpep_pickup_datetime').dt.strftime('%A').alias('pickup_day_of_week')
])

enriched.select(['trip_duration_minutes', 'trip_speed_mph', 'pickup_hour', 'pickup_day_of_week']).head()

trip_duration_minutes,trip_speed_mph,pickup_hour,pickup_day_of_week
f64,f64,i8,str
19.8,0.086869,0,"""Monday"""
6.6,0.272727,0,"""Monday"""
17.916667,0.262326,0,"""Monday"""
8.3,0.168675,0,"""Monday"""
6.1,0.131148,0,"""Monday"""


**6.** SGL Analysis

In [8]:
enriched = (enriched
                .join(zones_df, left_on='PULocationID', right_on='LocationID', how='left')
                .rename({'Zone': 'pickup_zone', 'Borough': 'pickup_borough'})
                .join(zones_df, left_on='DOLocationID', right_on='LocationID', how='left')
                .rename({'Zone': 'dropoff_zone', 'Borough': 'dropoff_borough'})
                # .drop(['LocationID'])
)

enriched.select([
    'PULocationID', 'pickup_zone', 'pickup_borough',
    'DOLocationID', 'dropoff_zone', 'dropoff_borough',
    'trip_distance', 'fare_amount', 'trip_duration_minutes'
]).head()

PULocationID,pickup_zone,pickup_borough,DOLocationID,dropoff_zone,dropoff_borough,trip_distance,fare_amount,trip_duration_minutes
i32,str,str,i32,str,str,f64,f64,f64
186,"""Penn Station/Madison Sq West""","""Manhattan""",79,"""East Village""","""Manhattan""",1.72,17.7,19.8
140,"""Lenox Hill East""","""Manhattan""",236,"""Upper East Side North""","""Manhattan""",1.8,10.0,6.6
236,"""Upper East Side North""","""Manhattan""",79,"""East Village""","""Manhattan""",4.7,23.3,17.916667
79,"""East Village""","""Manhattan""",211,"""SoHo""","""Manhattan""",1.4,10.0,8.3
211,"""SoHo""","""Manhattan""",148,"""Lower East Side""","""Manhattan""",0.8,7.9,6.1


In [9]:
import duckdb
con = duckdb.connect()

m) Top 10 pickup zones

In [10]:
busy = con.execute('''
  SELECT
    PULocationID as pickup_ID,
    pickup_zone as pickup_zone,
    COUNT(*) as zone_pickups
  FROM enriched
  GROUP BY PULocationID, pickup_zone
  ORDER BY zone_pickups DESC
  LIMIT 10
  ''').fetchdf()

print('List of busiest pickup zones')
print(busy)

List of busiest pickup zones
   pickup_ID                   pickup_zone  zone_pickups
0        161                Midtown Center        140137
1        237         Upper East Side South        140116
2        132                   JFK Airport        138201
3        236         Upper East Side North        133960
4        162                  Midtown East        104340
5        230     Times Sq/Theatre District        102954
6        186  Penn Station/Madison Sq West        102147
7        142           Lincoln Square East        101792
8        138             LaGuardia Airport         87663
9        239         Upper West Side South         86465


n) average fare per hour

In [11]:
h_avg=con.execute('''
  SELECT HOUR (tpep_pickup_datetime) AS hour,
  ROUND(AVG(fare_amount), 2) AS avg_fare
  FROM enriched
  GROUP BY hour
  ORDER BY hour
''').fetch_df()

print('Average fare per hour')
print(h_avg)

Average fare per hour
    hour  avg_fare
0      0     19.66
1      1     17.69
2      2     16.60
3      3     18.51
4      4     23.38
5      5     27.40
6      6     21.99
7      7     18.70
8      8     17.80
9      9     17.93
10    10     18.02
11    11     17.60
12    12     17.79
13    13     18.38
14    14     19.25
15    15     19.06
16    16     19.42
17    17     18.09
18    18     16.99
19    19     17.61
20    20     18.02
21    21     18.26
22    22     19.08
23    23     20.20


o) percentage payment types

In [12]:
pay_type = con.execute('''
  SELECT (payment_type) AS payment_type,
    CASE payment_type
      WHEN 0 THEN 'Credit card'
      WHEN 1 THEN 'Cash'
      WHEN 2 THEN 'No charge'
      WHEN 3 THEN 'Dispute'
      WHEN 4 THEN 'Unknown'
    END AS payment_description,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM enriched), 2) AS percentage
  FROM enriched
  GROUP BY payment_type
  ORDER BY payment_type
''').fetch_df()

print('Percentage of payment types')
print(pay_type)

Percentage of payment types
   payment_type payment_description  percentage
0             0         Credit card        4.01
1             1                Cash       80.10
2             2           No charge       14.73
3             3             Dispute        0.37
4             4             Unknown        0.79


p) average credit card tip percentage

In [13]:
cred_tip = con.execute('''
  SELECT DAYNAME(tpep_pickup_datetime) AS day_of_week,
    ROUND(AVG(tip_amount / fare_amount) * 100, 2) AS avg_tip_percentage
  FROM enriched
  WHERE payment_type = 0
  GROUP BY day_of_week
''').fetch_df()

print('Average credit card tip percentage')
print(cred_tip)

Average credit card tip percentage
  day_of_week  avg_tip_percentage
0      Friday               10.63
1    Saturday                7.55
2      Sunday                6.82
3      Monday                8.58
4     Tuesday                9.77
5   Wednesday               10.71
6    Thursday               10.47


q) most common pickup,dropoff pairs

In [14]:
pu_do_pair = con.execute('''
  SELECT
    PULocationID as pickup_ID,
    pickup_zone as pickup_zone,
    DOLocationID as dropoff_ID,
    dropoff_zone as dropoff_zone,
    COUNT(*) as pair_count
  FROM enriched
  GROUP BY PULocationID, pickup_zone, DOLocationID, dropoff_zone
  ORDER BY pair_count DESC
  LIMIT 5
''').fetch_df()

print('Most common pickup,dropoff pairs')
print(pu_do_pair)

Most common pickup,dropoff pairs
   pickup_ID            pickup_zone  dropoff_ID           dropoff_zone  \
0        237  Upper East Side South         236  Upper East Side North   
1        236  Upper East Side North         237  Upper East Side South   
2        236  Upper East Side North         236  Upper East Side North   
3        237  Upper East Side South         237  Upper East Side South   
4        161         Midtown Center         237  Upper East Side South   

   pair_count  
0       21641  
1       19199  
2       15192  
3       14112  
4       10139  


#Part 3: Dashboard Development

Additional imports for visualisation

In [15]:
# !pip install pandas numpy matplotlib seaborn plotly streamlit altair pyarrow

In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Plotly default template
import plotly.io as pio
pio.templates.default = 'plotly_white'

print('Libraries loaded!')

vis_sam = enriched.sample(n=100000, seed = 42)

Libraries loaded!


**8)** Key metrics Display

**9)** Required Visualisations

r) Bar chart

In [17]:
zone_counts_vis = (vis_sam
    .group_by('pickup_zone')
    .agg(pl.len().alias('zone_pickups'))
    .sort('zone_pickups', descending=True)
    .head(10)
)

fig1 = px.bar(zone_counts_vis,
              x = 'pickup_zone',
              y = 'zone_pickups',
              title = 'Top 10 Pickup Zones for NYC Taxies',
              labels = {'pickup_zone': 'Pickup Zone', 'zone_cnunts': 'Number of Trips'},
              text = 'zone_pickups'
              )

fig1.show()

This visualisation reveals that the popularito of pickup zones tend to happen in groups. It should also be noted that, minus the JFK Airport, the pickup zone groups tend to happen within localised areas

s) Line chart

In [18]:
avg_fare_vis = (vis_sam
    .group_by('pickup_hour')
    .agg(
        pl.mean('fare_amount').round(2).alias('avg_fare'),
    )
    .sort('pickup_hour')
)

fig2 = px.line(
    avg_fare_vis,
    x='pickup_hour',
    y='avg_fare',
    title='Average Fare per Hour',
    labels=({'pickup_hour': 'Hour', 'avg_fare': 'Average Fare'}),
    markers=True
)

fig2.show()

From the average fair per hour, it can be seen that there are spikes in the fare around when would be busiest. This would be a result of more people attempting to move around the city (i.e to work/school and vice versa), resulting in a reduction of supply and a spike in demand, this pattern can also be seen around midnight when supply would usually be low.

**t)** Histogram

In [19]:
fig3 = px.histogram(
    vis_sam,
    x = 'trip_distance',
    nbins = 50,
    range_x=[enriched['trip_distance'].min(), enriched['trip_distance'].max()],
    title = 'Distribution of Trip Distances',
    labels = {'trip_distance': 'Trip Distance (miles)', 'count': 'Number of Trips'}
)

fig3.show()

From the Histogram it can be seen that a majority of the Trips taken in NYC are between 0.1 to 3 miles. confirming the use case of taxies being used as last mile transport

**u)** Bar Chart

In [36]:
payment_lookup = pl.DataFrame({
    'payment_type': [0, 1, 2, 3, 4],
    'payment_description': ['Credit Card', 'Cash', 'No Charge', 'Dispute', 'Unknown']
})

pay_type_vis = (vis_sam
    .group_by('payment_type')
    .agg(pl.len().alias('count'))
    .with_columns(
        (pl.col('count') * 100.0 / pl.sum('count')).round(2).alias('percentage')
    )
    .sort('percentage', descending=True)
)

pay_type_vis = pay_type_vis.join(
    payment_lookup,
    on='payment_type',
    how='left'
).drop('payment_type')


print(pay_type_vis)

fig4 = px.bar(
    pay_type_vis,
    x='payment_description',
    y='percentage',
    title='Percentage of Payment Types',
    labels={'payment_description': 'Payment Type', 'percentage': 'Percentage'},
    text='percentage'
)

fig4.show()

shape: (5, 3)
┌───────┬────────────┬─────────────────────┐
│ count ┆ percentage ┆ payment_description │
│ ---   ┆ ---        ┆ ---                 │
│ u32   ┆ f64        ┆ str                 │
╞═══════╪════════════╪═════════════════════╡
│ 80022 ┆ 80.02      ┆ Cash                │
│ 14765 ┆ 14.76      ┆ No Charge           │
│ 4041  ┆ 4.04       ┆ Credit Card         │
│ 829   ┆ 0.83       ┆ Unknown             │
│ 343   ┆ 0.34       ┆ Dispute             │
└───────┴────────────┴─────────────────────┘


The above bar chart shows that ta massive majority NYC Taxi trips are paid for with cash. wuth the followup being no charge.

**v)** Heatmap

In [37]:
fig5 = px.density_heatmap(
    vis_sam,
    x = 'pickup_hour',
    y = 'pickup_day_of_week',
    title = 'Trip Density by Hour and Day of Week',
    labels = {'pickup_hour': 'Pickup Hour', 'pickup_day_of_week': 'Pickup Day of Week', 'count': 'Number of Trips'},
    color_continuous_scale = 'agsunset',
    category_orders = {'pickup_day_of_week': ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']}

)

fig5.show()

This density heatmap sgows that during the week there are minimal trips taken in the early morning, with an increase in trips when the work day would be starting. With the most amount of trips tak4nafter the work day ends presumably for a mojority of people to go home.